**Python version:**  3.x

In [1]:
# set the path of the directories where your modules and files are
modules_path = 'C:\\Users\loren\Documents\Progetti\__repo__\_Modules_LF95'
files_path = 'C:\\Users\loren\Documents\Progetti\Scuola\DataMining_Advanced\_files'

#### Moduli

In [2]:
import os
import sys

if os.path.isdir(modules_path) and (modules_path not in sys.path):
    sys.path.append(modules_path)

files_list = os.listdir(files_path)
print('available files for the project:', files_list, sep='\n')

%matplotlib inline

from data_science.preprocessing import df_handle
from data_science.preprocessing import data_understanding
from data_science.preprocessing import data_preparation
from data_science.modeling import unsupervised_learning
from data_science.evaluation import unsupervised_evaluation

Available Modules for the project:
['data_preparation.py', 'data_understanding.py', 'df_handle.py', 'modeling.py', 'model_evaluation.py', '__pycache__']

Available Files for the project:
['datatest.txt', 'datatest2.txt', 'datatraining.txt', 'test1.csv', 'test2.csv', 'training.csv']


In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

creiamo un validation set con il 30% del training set

In [4]:
temp1, X_val, temp2, y_val = df_handle.ml_setup(trainFile_name='training.csv', y_name='Occupancy', test_portion=0.3,
                                                    search_in_folder=files_path, date_col_name='date')

In [5]:
del temp1, temp2

importiamo training e test veri e propri

In [6]:
X_train, X_test, y_train, y_test = df_handle.ml_setup(trainFile_name='training.csv', y_name='Occupancy',
                                                      testFile_name='test1.csv', search_in_folder=files_path,
                                                      date_col_name='date')

In [7]:
#del X_train['year'], X_train['month'], X_train['day'], X_train['hour'], X_train['minute'], X_train['second']
#del X_test['year'], X_test['month'], X_test['day'], X_test['hour'], X_test['minute'], X_test['second']
#del X_val['year'], X_val['month'], X_val['day'], X_val['hour'], X_val['minute'], X_val['second']

In [8]:
#del X_train['Humidity'], X_train['HumidityRatio']
#del X_test['Humidity'], X_test['HumidityRatio']
#del X_val['Humidity'], X_val['HumidityRatio']

In [7]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = np.array(X_val).reshape((X_val.shape[0], X_val.shape[1], 1))

# Data Normalization

In [8]:
from tslearn.preprocessing import TimeSeriesScalerMinMax

In [9]:
scaler = TimeSeriesScalerMinMax()
#X = scaler.fit_transform(X_train).reshape(X_train.shape[0], X_train.shape[1])

# \Multivariate LSTM Classifier\

In [11]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, Conv2D, MaxPool2D, Flatten, Dropout, LeakyReLU, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [21]:
def build_lstm(n_timesteps, n_outputs, n_features):
    model = Sequential()
    model.add(LSTM(4, input_shape=(n_timesteps, n_features), return_sequences=True, 
                        kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    
    #1
    for _ in range(1):
        model.add(LSTM(4, kernel_initializer='TruncatedNormal', return_sequences=True))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.04))   

    #2
    model.add(LSTM(32, kernel_initializer='TruncatedNormal', return_sequences=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.7))
    
    #3
    for _ in range(1):
        model.add(Dense(256, kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.2))
    #4
    for _ in range(1):
        model.add(Dense(64, kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.7))

    #5
    model.add(Dense(32, kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.4))
        
    model.add(Dense(n_outputs, activation='sigmoid'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [22]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_train)), X_train.shape[2] 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)
print("N. FEATURES: ", n_features)

TIMESTEPS:  11
N. LABELS:  2
N. FEATURES:  1


In [23]:
lstm = build_lstm(n_timesteps, n_outputs, n_features)

In [24]:
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 11, 4)             96        
_________________________________________________________________
batch_normalization_9 (Batch (None, 11, 4)             16        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 11, 4)             0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 11, 4)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 11, 4)             144       
_________________________________________________________________
batch_normalization_10 (Batc (None, 11, 4)             16        
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 11, 4)            

In [25]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [26]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [27]:
history_lstm = lstm.fit(X_train, y_train, epochs=1, batch_size=mini_batch_size, callbacks=callbacks,
                       validation_data=(X_val, y_val)).history

Train on 8143 samples, validate on 2443 samples
Epoch 1/1
8143/8143 [==============================] - 11s 1ms/step - loss: 0.2649 - accuracy: 0.8979 - val_loss: 0.8597 - val_accuracy: 0.7876


In [28]:
y_pred = np.argmax(lstm.predict(X_test), axis=1)

print('Accuracy %s' % model_evaluation.accuracy_score(y_test, y_pred))
print('F1-score %s' % model_evaluation.f1_score(y_test, y_pred, average=None))
print(model_evaluation.classification_report(y_test, y_pred))

Accuracy 0.6352720450281426
F1-score [0.77696191 0.        ]
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1693
           1       0.00      0.00      0.00       972

    accuracy                           0.64      2665
   macro avg       0.32      0.50      0.39      2665
weighted avg       0.40      0.64      0.49      2665



In [20]:
lstm.evaluate(X_test, y_test)

2665/2665 [==============================] - 0s 180us/step


[1.33861720873983, 0.6352720260620117]